In [13]:
import numpy as np
import pickle

from inv_kin_body2foot import *

In [14]:
class FootInfo():
    def __init__(self, time_vec, foot_left, foot_right):
        self.time_vec = time_vec
        self.foot_left = foot_left
        self.foot_right = foot_right

with open('foot_infos', 'rb') as foot_infos_file:
    foot_infos = pickle.load(foot_infos_file)

In [15]:
n = np.array([[0], [0], [-1]])
s = np.array([[-1], [0], [0]])
a = np.array([[0], [1], [0]])
R = np.block([n, s, a])

for fidx in range(len(foot_infos)):
    statel = foot_infos[fidx].foot_left[np.ix_([0, 2, 4]), :]
    stater = foot_infos[fidx].foot_right[np.ix_([0, 2, 4]), :]
    
    num_idx = statel.shape[1]
    joints_left = np.zeros((6, num_idx))
    joints_right = np.zeros((6, num_idx))
    trans_mat_left = np.zeros((4, 4, num_idx))
    trans_mat_right = np.zeros((4, 4, num_idx))
    
    for sidx in range(num_idx):
        p = statel[:, sidx]
        trans_mat = np.block([[R, p],[0, 0, 0, 1]])
        is_left = true
        q_left = inv_kin_body2foot(trans_mat, is_left)
        joints_left[:, sidx]

array([[ 0, -1,  0,  0],
       [ 0,  0,  1,  0],
       [-1,  0,  0, -1],
       [ 0,  0,  0,  1]])